In [1]:
# Code to to explore and reconcile outline inventories
#
# Written 2023-07-11 by W. Sauthoff (wsauthoff.github.io)

In [2]:
# Import packages
import fiona
import geopandas as gpd
import h5py
import numpy as np
import pandas as pd
from pyproj import CRS, Geod, Transformer
from shapely.geometry import Polygon, MultiPolygon

# Define utility functions
def find_intersections(gdf1, gdf2):
    # Create an empty list to store the results
    intersections = []

    # Iterate over each geometry in gdf1
    for index1, geom1 in gdf1.geometry.items():
        # Compare with each geometry in gdf2
        for index2, geom2 in gdf2.geometry.items():
            if geom1.intersects(geom2):
                # If they intersect, add the indices to the list
                intersections.append((index1, index2))
    return intersections

# Create a Geod object for calculating area on the WGS84 ellipsoid
geod = Geod(ellps="WGS84")

In [3]:
# Import active subglacial lake outlines from Smith and others (2009) (S09)
# As released in Smith and others, 2012 dataset (https://doi.org/10.15784/601439)
fiona.drvsupport.supported_drivers['KML'] = 'rw'
S09_outlines_lonlat = gpd.read_file('/home/jovyan/1_outlines_candidates/Sauthoff-2024-J.Glaciol./input/lake_outlines/Smith2009_lakes/Antarctic_lakes.kml', driver='KML')

# Ensure GeoDataFrame is in EPSG:4326 for geodesic area calculation
if S09_outlines_lonlat.crs != 'EPSG:4326':
    S09_outlines_lonlat = S09_outlines_lonlat.to_crs('EPSG:4326')

# Calculate the geodesic area for each polygon
S09_outlines_lonlat['area (m^2)'] = S09_outlines_lonlat['geometry'].apply(    
    lambda poly: abs(geod.polygon_area_perimeter(
    poly.exterior.coords.xy[0], poly.exterior.coords.xy[1])[0]) if poly is not None and poly.is_valid else None)

# Convert to crs to epsg:3031
S09_outlines = S09_outlines_lonlat.to_crs(3031)

# Delete original geodataframe
del S09_outlines_lonlat

# Display geodataframe
S09_outlines

/srv/conda/envs/notebook/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08
...,...,...,...,...
119,Whillans_6,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-451544.869 -488823.261 1.000, -45...",7.458477e+07
120,Whillans_7,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-543163.376 -500759.165 1.000, -54...",7.696570e+07
121,Whillans_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-654478.748 -281124.560 1.000, -65...",1.625714e+08
122,Wilkes_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((2214185.180 -666018.604 1.000, 221...",5.880773e+08


In [4]:
S09_outlines

,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08
...,...,...,...,...
119,Whillans_6,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-451544.869 -488823.261 1.000, -45...",7.458477e+07
120,Whillans_7,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-543163.376 -500759.165 1.000, -54...",7.696570e+07
121,Whillans_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-654478.748 -281124.560 1.000, -65...",1.625714e+08
122,Wilkes_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((2214185.180 -666018.604 1.000, 221...",5.880773e+08


In [5]:
# Import active subglacial lake outlines from Siegfried & Fricker (2018) (SF18)
# Original pub: https://doi.org/10.1017/aog.2017.36 
# Code for loading lake outlines available in code bank associated with Siegfried & Fricker (2021), https://doi.org/10.1029/2020GL091089: 
# https://github.com/mrsiegfried/Siegfried2021-GRL/blob/main/data/outlines/load_lakes.ipynb

# import subglacial lake outlines (Siegfried & Fricker, 2018)
h5f = h5py.File('/home/jovyan/1_outlines_candidates/Sauthoff-2024-J.Glaciol./input/lake_outlines/SiegfriedFricker2018_lakes/SiegfriedFricker2018-outlines.h5', 'r')
outline_geometries = [] # store polygons
citations = [] # store citation information

# we're going to calculate geodesic lake area because that is often screwed up
# and occasionally incorrect in the literature
areas = []

# we're going to need to do some coordinate transforms for the geodesic area
# define CRS for Antarcica and make a converter from xy to ll
CRS_LL = "EPSG:4326" # wgs84 in lon,lat
CRS_XY = h5f.attrs.get('proj_crs') # get projection from hdf5 file
XY_TO_LL = Transformer.from_crs(CRS_XY, CRS_LL, always_xy = True) # make coord transformer
geod = CRS(CRS_LL).get_geod() # geod object for calculating geodesic area on defined ellipsoid

# look through each lake and load all of it's info
for lake in h5f.keys():
    outline_x = h5f[lake]['x'][:]
    outline_y = h5f[lake]['y'][:]
    outlines_xy = np.stack((outline_x, outline_y),axis=2).reshape(outline_x.shape[1], 2)

    # A single lake with multiple polygons is NaN broken---need to identify and
    # load as a MultiPolygon. Otherwise it's easy (just load as polygon)
    if np.isnan(outlines_xy)[:,0].sum() == 0:
        geometry = Polygon(outlines_xy)
        lon, lat = XY_TO_LL.transform(outlines_xy[:,0], outlines_xy[:,1])
        this_area = abs(geod.polygon_area_perimeter(lon,lat)[0])
    else:
        this_area = 0
        # break at NaN values and load each as separate polygons
        idx = np.where(np.isnan(outlines_xy[:,0]))[0]

        # grab outline of first lake before getting into the loop
        this_outline = outlines_xy[0:idx[0],:]
        pgons = [Polygon(this_outline)] # put the first polygon in a list
        lon,lat = XY_TO_LL.transform(this_outline[:,0], this_outline[:,1])
        this_area += abs(geod.polygon_area_perimeter(lon,lat)[0])/1e6 # add its area
        for i in np.arange(0,len(idx)):
            if i == len(idx)-1:
                this_outline = outlines_xy[idx[i]+1:,:]
            else:
                this_outline = outlines_xy[idx[i]+1:idx[i+1]]

            pgons.append(Polygon(this_outline))
            lon,lat = XY_TO_LL.transform(this_outline[:,0], this_outline[:,1])
            this_area += abs(geod.polygon_area_perimeter(lon,lat)[0])/1e6
        geometry = MultiPolygon(pgons)

    # append all the results in the right place
    outline_geometries.append(geometry)
    citations.append(h5f[lake].attrs.get('citation')[0].decode('UTF-8'))
    areas.append(this_area)

# make a pandas dataframe with all the necessary info
df = pd.DataFrame(zip(h5f.keys(), outline_geometries, areas, citations),
                  columns=['name', 'geometry', 'area (m^2)', 'cite'])
# convert to geopands geodataframe
SF18_outlines = gpd.GeoDataFrame(df, crs=CRS_XY, geometry=outline_geometries)
# close HDF5 file
h5f.close()

# Display geodataframe
SF18_outlines

,name,geometry,area (m^2),cite
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [6]:
for idx in range(len(SF18_outlines['cite'].unique())): 
    length = len(SF18_outlines[SF18_outlines['cite'].str.contains(SF18_outlines['cite'].unique()[idx])])
    print(length, 'lakes from', SF18_outlines['cite'].unique()[idx])

97 lakes from Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879
7 lakes from Fricker & Scambos, 2009, J. Glac., doi:10.3189/002214309788608813
1 lakes from McMillan and others, 2013, GRL, doi:10.1002/grl.50689
3 lakes from Kim and others, 2016, TC, doi:10.5194/tc-10-2971-2016
1 lakes from Carter and others, 2013, J. Glac., doi:10.3189/2013JoG13J085
8 lakes from Fricker and others, 2010, J. Glac., doi:10.3189/002214310791968557
9 lakes from Fricker and others, 2014, J. Glac., doi:10.3189/2014JoG14J063
1 lakes from Siegfried & Fricker, 2018, Ann. Glac., doi:10.1017/aog.2017.36
4 lakes from Smith and others, 2017, TC, doi:10.5194/tc-11-451-2017


In [7]:
# Use a regular expression to extract the year from the string in 'your_column'
# The extracted year is added as a new column 'year'
SF18_outlines['year'] = SF18_outlines['cite'].str.extract('(20\d{2})')

# Convert the year column to numeric to ensure proper sorting
SF18_outlines['year'] = pd.to_numeric(SF18_outlines['year'])

In [8]:
SF18_outlines[SF18_outlines['cite'] != 'Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879'].sort_values('year')

,name,geometry,area (m^2),cite,year
23,ConwaySubglacialLake,"POLYGON ((-312825.002 -511425.001, -312699.997...",2.669973e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
93,MercerSubglacialLake,"POLYGON ((-299950.018 -500675.034, -299949.971...",1.432030e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
123,UpperSubglacialLakeConway,"POLYGON ((-370200.008 -533925.036, -370200.008...",1.866440e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
81,Lake12,"POLYGON ((-224449.970 -604674.969, -224449.970...",6.689842e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
80,Lake10,"POLYGON ((-227949.981 -569549.997, -228075.000...",2.587121e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
125,WhillansSubglacialLake,"POLYGON ((-284324.993 -560675.019, -284200.010...",6.196906e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
41,EngelhardtSubglacialLake,"POLYGON ((-271824.984 -628674.969, -271699.995...",3.577632e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
92,Mac8,"POLYGON ((-842816.190 -708487.890, -842383.070...",1.072318e+08,"Fricker and others, 2010, J. Glac., doi:10.318...",2010
85,Mac1,"POLYGON ((-629350.010 -889125.010, -629325.010...",1.563754e+08,"Fricker and others, 2010, J. Glac., doi:10.318...",2010
86,Mac2,"POLYGON ((-657250.020 -890550.040, -657249.980...",1.457161e+08,"Fricker and others, 2010, J. Glac., doi:10.318...",2010


In [9]:
# Siegfried and Fricker (2018) only contains 97 Smith and others (2009) citations due to 27 redelineations: 
# 124 lakes in Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879
# -7 Fricker & Scambos, 2009, J. Glac., doi:10.3189/002214309788608813 redelineates Mercer/Whillans lakes
# -8 Fricker and others, 2010, J. Glac., doi:10.3189/002214310791968557 redelineates MacAyeal Ice Stream lakes
# -1 Carter and others, 2013, J. Glac., doi:10.3189/2013JoG13J085 redelineates Mercer_1 as Lake78
# -9 Fricker and others, 2014, J. Glac., doi:10.3189/2014JoG14J063 redelineates Recovery Glacier lakes
# -2 Siegfried & Fricker, 2018, Ann. Glac., doi:10.1017/aog.2017.36 redelineates Slessor2 and Slessor3 as Slessor23
# 97 citations to Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879 in Siegfried and Fricker, 2018 inventory

In [10]:
# Read subglacial lake point data from Livingstone and others (2022) (L22), https://doi.org/10.1038/s43017-021-00246-9
url = 'https://static-content.springer.com/esm/art%3A10.1038%2Fs43017-021-00246-9/MediaObjects/43017_2021_246_MOESM1_ESM.xlsx'
use_cols = ['Name', 'Lat.  oN', 'Lon. oE', 'Lake Type', 'References', 'Prior Inventory']
import_rows = np.arange(0,676)
L22_points = pd.read_excel(url, sheet_name='Antarctica', usecols=use_cols, skiprows = lambda x: x not in import_rows)

# View just the active lakes of the pandas dataset
L22_activelake_points = L22_points[L22_points['Lake Type'] == 'Active']

# Reset the index, dropping the old one
L22_activelake_points = L22_activelake_points.reset_index(drop=True)

# Display geodataframe
L22_activelake_points

,Name,Lat. oN,Lon. oE,Lake Type,References,Prior Inventory
0,Bindschadler5,-80.610,-123.994,Active,Gray et al. (2005); Smith et al. (2009),Wright & Siegert (2012)
1,Kamb10,-81.446,-120.134,Active,Gray et al. (2005); Smith et al. (2009),Wright & Siegert (2012)
2,L1,-74.000,133.270,Active,Wingham et al. (2006); Surveyed by ICECAP in 2...,Wright & Siegert (2012)
3,U1,-75.940,135.000,Active,Wingham et al. (2006),Wright & Siegert (2012)
4,U2,-76.340,135.770,Active,Wingham et al. (2006),Wright & Siegert (2012)
...,...,...,...,...,...,...
134,THW142,-76.618,-106.185,Active,Smith et al. (2017); Hoffman et al. (2020); Ma...,NaN
135,THW124,-76.462,-106.893,Active,Smith et al. (2017) and Hoffman et al. (2020).,NaN
136,THW70,-75.990,-107.023,Active,Smith et al. (2017) and Hoffman et al. (2020).,NaN
137,WT,-77.300,-114.000,Active,Hoffman et al. (2020).,NaN


In [11]:
for idx in range(len(L22_activelake_points['References'].unique())): 
    length = len(L22_activelake_points[L22_activelake_points['References'].str.contains(L22_activelake_points['References'].unique()[idx], regex=False)])
    print(length, 'lakes from', L22_activelake_points['References'].unique()[idx])
    print()

2 lakes from Gray et al. (2005); Smith et al. (2009)

1 lakes from Wingham et al. (2006); Surveyed by ICECAP in 2008/09 (Wright et al., 2012)

4 lakes from Wingham et al. (2006)

1 lakes from Fricker et al. (2007), Blankenship et al. (2009), Siegfried et al., (2014), Siegfried & Fricker (2018)

1 lakes from Fricker et al. (2007); Siegfried et al. (2014); Siegfried & Fricker (2018)

1 lakes from Lake Engelhardt (Fricker et al., 2007); Carter et al., (2013); Siegfried & Fricker (2018)

2 lakes from Fricker et al. (2007); Siegfried & Fricker (2018)

1 lakes from Lake Whillans, (Fricker et al., 2007); Geophysical observations made at low stand (Christianson et al., 2012 and Horgan et al. 2012); Carter et al., (2013); Siegfried et al. (2014); Siegfried & Fricker (2018).

1 lakes from Lake Conway (Fricker et al., 2007); Carter et al., (2013); Siegfried et al. (2014); Siegfried & Fricker (2018)

2 lakes from Stearns et al., (2008)

114 lakes from Smith et al. (2009)

1 lakes from Smith et al.

In [12]:
for idx in range(len(L22_activelake_points['References'].unique())):
    # Filter the DataFrame for the current unique reference
    current_reference = L22_activelake_points['References'].unique()[idx]
    filtered_df = L22_activelake_points[L22_activelake_points['References'].str.contains(current_reference, regex=False)]
    
    # Print the length and reference name
    length = len(filtered_df)
    print(length, 'lakes from', current_reference)
    
    # Print the names associated with the current reference
    print("Name(s):", ", ".join(filtered_df['Name']))

2 lakes from Gray et al. (2005); Smith et al. (2009)
Name(s): Bindschadler5, Kamb10
1 lakes from Wingham et al. (2006); Surveyed by ICECAP in 2008/09 (Wright et al., 2012)
Name(s): L1
4 lakes from Wingham et al. (2006)
Name(s): L1, U1, U2, U3
1 lakes from Fricker et al. (2007), Blankenship et al. (2009), Siegfried et al., (2014), Siegfried & Fricker (2018)
Name(s): PPT-16             (Lake Mercer) (Mercer2)
1 lakes from Fricker et al. (2007); Siegfried et al. (2014); Siegfried & Fricker (2018)
Name(s): Mercer1
1 lakes from Lake Engelhardt (Fricker et al., 2007); Carter et al., (2013); Siegfried & Fricker (2018)
Name(s): Whillans1          (Lake Engelhardt)
2 lakes from Fricker et al. (2007); Siegfried & Fricker (2018)
Name(s): Whillans2a (L12), Whillans2b (L10)
1 lakes from Lake Whillans, (Fricker et al., 2007); Geophysical observations made at low stand (Christianson et al., 2012 and Horgan et al. 2012); Carter et al., (2013); Siegfried et al. (2014); Siegfried & Fricker (2018).
Name(

In [13]:
# View head of Smith and others, 2009 static outline geodataframe
S09_outlines.head()

,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08


In [14]:
S09_outlines[S09_outlines.Name == 'Recovery_8']

,Name,Description,geometry,area (m^2)
98,Recovery_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-74016.771 896653.746 1.000, -7350...",4.355089e+08


In [15]:
# Find outlines in SiegfriedFricker2018_outlines static outline geodataframe with Smith and others, 2009 citation
# These lakes have not been redefined with a new outline and can be analysed as is
SF18_outlines_S09only = SF18_outlines[SF18_outlines['cite'].isin(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879'])]
SF18_outlines_S09only

,name,geometry,area (m^2),cite,year
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
...,...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009


In [16]:
# Find outlines in SiegfriedFricker2018_outlines geodataframe with citations not matching Smith and others, 2009
# These are eitherly newly identified lakes or lakes that have been redefined with new static outlines 
SF18_outlines_SF18only = SF18_outlines[~SF18_outlines['cite'].isin(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879'])]
SF18_outlines_SF18only

,name,geometry,area (m^2),cite,year
23,ConwaySubglacialLake,"POLYGON ((-312825.002 -511425.001, -312699.997...",2.669973e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
25,Cook_E2,"POLYGON ((765592.392 -1714713.856, 765682.174 ...",2.680933e+08,"McMillan and others, 2013, GRL, doi:10.1002/gr...",2013
41,EngelhardtSubglacialLake,"POLYGON ((-271824.984 -628674.969, -271699.995...",3.577632e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
65,KT1,"POLYGON ((-556189.687 -681400.000, -556000.000...",4.549884e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29...",2016
66,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29...",2016
67,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29...",2016
80,Lake10,"POLYGON ((-227949.981 -569549.997, -228075.000...",2.587121e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
81,Lake12,"POLYGON ((-224449.970 -604674.969, -224449.970...",6.689842e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189...",2009
82,Lake78,"MULTIPOLYGON (((-257325.009 -525800.047, -2571...",2.330027e+02,"Carter and others, 2013, J. Glac., doi:10.3189...",2013
85,Mac1,"POLYGON ((-629350.010 -889125.010, -629325.010...",1.563754e+08,"Fricker and others, 2010, J. Glac., doi:10.318...",2010


In [17]:
# Ensure that filtering worked properly
len(SF18_outlines) == len(SF18_outlines_S09only) + len(SF18_outlines_SF18only)

True

In [18]:
# Find spatial intersections between two static outline geodataframes to indicate groups of outlines that are redefined outlines of one lake
S09_SF18_intersections = find_intersections(S09_outlines, SF18_outlines)

In [19]:
# Extract intersecting indicies from each static outline geodataframe for later use
S09_intersection_indices = [inner_list[0] for inner_list in S09_SF18_intersections]
SF18_intersection_indices = [inner_list[1] for inner_list in S09_SF18_intersections]

In [20]:
# Store the lake names from Smith and others, 2009 and Siegfried & Fricker, 2018 that intersect
# Store 'name' column for rows in gdf1 corresponding to indices in indices1
S09_intersections_names = S09_outlines.iloc[S09_intersection_indices]['Name']

# Store 'name' column for rows in gdf2 corresponding to indices in indices2
SF18_intersections_names = SF18_outlines.iloc[SF18_intersection_indices]['name']

# Concatenate into dataframe
intersecting_lake_names = pd.concat([S09_intersections_names.reset_index(drop=True), SF18_intersections_names.reset_index(drop=True)], axis=1)
intersecting_lake_names.columns = ['S09', 'SF18']

# Filtering rows where column1 and column2 differ indicating lake was redelineated
intersecting_lake_names_diff = intersecting_lake_names[intersecting_lake_names['S09'] != intersecting_lake_names['SF18']]

In [21]:
print(intersecting_lake_names.to_string())

                S09                       SF18
0    Bindschadler_1             Bindschadler_1
1    Bindschadler_1                       Mac7
2    Bindschadler_2             Bindschadler_2
3    Bindschadler_2                       Mac8
4    Bindschadler_3             Bindschadler_3
5    Bindschadler_4             Bindschadler_4
6    Bindschadler_5             Bindschadler_5
7    Bindschadler_6             Bindschadler_6
8            Byrd_1                     Byrd_1
9            Byrd_2                     Byrd_2
10         Byrd_s10                   Byrd_s10
11         Byrd_s11                   Byrd_s11
12         Byrd_s12                   Byrd_s12
13         Byrd_s13                   Byrd_s13
14         Byrd_s14                   Byrd_s14
15         Byrd_s15                   Byrd_s15
16          Byrd_s1                    Byrd_s1
17          Byrd_s2                    Byrd_s2
18          Byrd_s3                    Byrd_s3
19          Byrd_s4                    Byrd_s4
20          B

In [22]:
print(intersecting_lake_names_diff)

                S09                       SF18
1    Bindschadler_1                       Mac7
3    Bindschadler_2                       Mac8
59     InstituteE_1               Institute_E1
60     InstituteE_2               Institute_E2
61     InstituteW_1               Institute_W1
62     InstituteW_2               Institute_W2
63      KambTrunk_1                        KT1
78       Macayeal_1                       Mac1
79       Macayeal_2                       Mac2
80       Macayeal_3                       Mac4
81       Macayeal_3                       Mac5
82       Macayeal_4                       Mac6
83         Mercer_1                     Lake78
84         Mercer_2       MercerSubglacialLake
92      Recovery_10                       Rec9
93      Recovery_11                      Rec10
94       Recovery_1                       Rec1
95       Recovery_2                       Rec1
96       Recovery_3                       Rec2
97       Recovery_4                       Rec3
98       Reco

In [37]:
# View rows with intersection between the S09 static outlines and the SF18 static outlines
S09_intersections = S09_outlines[S09_outlines.index.isin(S09_intersection_indices)] 
S09_intersections

,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08
...,...,...,...,...
119,Whillans_6,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-451544.869 -488823.261 1.000, -45...",7.458477e+07
120,Whillans_7,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-543163.376 -500759.165 1.000, -54...",7.696570e+07
121,Whillans_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-654478.748 -281124.560 1.000, -65...",1.625714e+08
122,Wilkes_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((2214185.180 -666018.604 1.000, 221...",5.880773e+08


In [38]:
# Compare lengths of geodataframes
print(len(S09_outlines))
print(len(S09_intersections))

124
123


In [25]:
# Extract rows with no intersection between the S09 and the SF18 renamed or redefined static outlines
# These are new lakes that were delineated by Smith and others, 2009 and left as is (not redelineated/renamed)
# These lakes only have one static outline and can be analysed as is
S09_non_intersections = S09_outlines[~S09_outlines.index.isin(S09_intersection_indices)] 
S09_non_intersections

,Name,Description,geometry,area (m^2)
98,Recovery_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-74016.771 896653.746 1.000, -7350...",4.355089e+08


In [24]:
# Do the same for the SF18 static outlines
SF18_intersections = SF18_outlines[SF18_outlines.index.isin(SF18_intersection_indices)] 
SF18_intersections

,name,geometry,area (m^2),cite,year
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
...,...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",2009


In [26]:
# Extract rows with no intersection between the Smith and others, 2009 and the renamed or redefined static outlines in Siegfried & Fricker, 2018
# These are new lakes that were added by publications subsequent to Smith and others, 2009
# These lakes only have one static outline and can be analysed as is
SF18_non_intersections = SF18_outlines[~SF18_outlines.index.isin(SF18_intersection_indices)] 
SF18_non_intersections

,name,geometry,area (m^2),cite,year
66,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29...",2016
67,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29...",2016
87,Mac3,"POLYGON ((-642325.010 -883925.000, -642449.980...",1.502500e+08,"Fricker and others, 2010, J. Glac., doi:10.318...",2010
117,Thw_124,"POLYGON ((-1419402.000 -431819.000, -1421479.0...",5.761346e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-...",2017
118,Thw_142,"POLYGON ((-1404705.000 -410106.000, -1404692.0...",1.607689e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-...",2017
119,Thw_170,"POLYGON ((-1380997.000 -398714.000, -1382313.0...",1.940960e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-...",2017
120,Thw_70,"POLYGON ((-1476557.000 -449098.000, -1474606.0...",3.540275e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-...",2017


In [27]:
# Ensure filtering worked properly
len(S09_outlines) == len(S09_intersections) + len(S09_non_intersections)

True

In [28]:
len(S09_intersections)

123

In [29]:
len(S09_non_intersections)  # This is Recovery_8 that was dropped in SF18 inventory

1

In [30]:
len(SF18_outlines) == len(SF18_intersections) + len(SF18_non_intersections)

True

In [31]:
len(SF18_intersections)  # Lakes from S09 inventory that were either keep as is or redelinated; there is one more than S09_intersections because 

124

In [32]:
len(SF18_non_intersections)  # These are new lakes added in the SF18 inventory from other publications since S09 inventory

7

In [33]:
# Create list of S09 lakes redelineated in SF18
S09_SF18_redelineated_lake_names = [   
    ['Cook_E2', 'Cook_E2'],
    ['KambTrunk_1', 'KT1'],
    ['Macayeal_1', 'Mac1'],
    ['Macayeal_2', 'Mac2'],
    ['Mercer_1', 'Lake78'],
    ['Mercer_2', 'MercerSubglacialLake'],
    ['Recovery_3', 'Rec2'],
    ['Recovery_4', 'Rec3'],
    ['Recovery_5', 'Rec4'], 
    ['Recovery_6', 'Rec5'],
    ['Recovery_7', 'Rec6'],
    ['Recovery_9', 'Rec8'],
    ['Recovery_10', 'Rec9'],
    ['Recovery_11', 'Rec10'],     
    ['Whillans_1', 'EngelhardtSubglacialLake'],            
    ['Whillans_2a', 'Lake12'],
    ['Whillans_2b', 'Lake10'],
    ['Whillans_3', 'WhillansSubglacialLake'],
    ['Whillans_4', 'ConwaySubglacialLake'],
    ['Whillans_5', 'UpperSubglacialLakeConway'],
    # Two S09 lakes converted to one SF18 lake
    [['Slessor_2', 'Slessor_3'], 'Slessor_23'],
    [['Recovery_1', 'Recovery_2'], 'Rec1'],
    # One S09 lake converted to two SF18 lakes
    ['Macayeal_3', ['Mac4', 'Mac5']]
]

In [34]:
len(SF18_outlines_SF18only)

34

In [35]:
len(SF18_outlines_S09only)

97

In [ ]:
# These are lakes that were defined in Smith and others, 2009 and subsequently renamed or 
# redelineated by publications after Smith and others, 2009
# We will analyze as a unified polygon of all past static outlines for each lake